In [ ]:
!pip install isbnlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 2.2 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import isbnlib
import matplotlib.pyplot as plt
from scipy.cluster.vq import kmeans, vq
from pylab import plot, show
from matplotlib.lines import Line2D
import matplotlib.colors as mcolors
from sklearn.cluster import KMeans
from sklearn import neighbors
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from progressbar import ProgressBar
from sklearn.metrics import silhouette_score
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")
print('Libraries imported!')

Libraries imported!


In [ ]:
df = pd.read_csv(r'/content/drive/MyDrive/AI Project/zerin/book_info.csv', error_bad_lines = False)
df = df.drop(columns=['Unnamed: 0'])
df

,id,Book_Name,Writer_Name,Category,Rating,Review,Site,sentiment,label,Book_name_without_spaces
0,96523,মুনতাখাব হাদীস,মাওলানা মোঃ সাখাওয়াত উল্লাহ,হাদিস ও সুন্নাত,1,(১) বই টি শত শত জাল হাদিসে ভরপুর । (২) দ্বীন ...,Rokomari,negative,0,মুনতাখাবহাদীস
1,39118,মেঘাচ্ছন্ন ঈমান (পেপারব্যাক),শাইখ আব্দুল মালিক আল কাসিম,"ঈমান, আক্বিদা ও তাওবাহ",1,"বই রিভিউ : মেঘাচ্ছন্ন ঈমান পাঠ অনুভূতি: ""একটি...",Rokomari,negative,0,মেঘাচ্ছন্নঈমান(পেপারব্যাক)
2,141682,সামষ্টিক অর্থনীতি (পার্স কোর্স দ্বিতীয় বর্ষের...,সুকেশ চন্দ্র জোয়ারদার,ডিগ্রি,1,পড়তে চাই,Rokomari,negative,0,সামষ্টিকঅর্থনীতি(পার্সকোর্সদ্বিতীয়বর্ষের-৩য়পত্...
3,136770,ছাত্রলীগের ইতিহাস বাংলাদেশের ইতিহাস (১৯৪৮-১৯৭...,সাইফুর রহমান সোহাগ,প্রসঙ্গ: বাংলাদেশের রাজনীতি,1,ঠিক আছে,Rokomari,negative,0,ছাত্রলীগেরইতিহাসবাংলাদেশেরইতিহাস(১৯৪৮-১৯৭০প্রথ...
4,82740,"এসএসসি বিজ্ঞান মেইড ইজি প্রশ্নপত্র, সকল শিক্ষ...",পাঞ্জেরী সম্পাদনা পর্ষদ (সম্পাদক),নবম ও দশম (এসএসসি): বিজ্ঞান বিভাগ পাঠ্য সহায়...,1,এটি এসএসসি শিক্ষার্থীর 2020 এর জন্য একটি পরীক্...,Rokomari,negative,0,"এসএসসিবিজ্ঞানমেইডইজিপ্রশ্নপত্র,সকলশিক্ষাবোর্ড,..."
...,...,...,...,...,...,...,...,...,...,...
6306,21943,"তিমির প্রেম, সপ্ত আতঙ্ক, লর্না ডুন (পেপারব্যা...",আর. ডি. ব্ল্যাকমোর,"রহস্য, গোয়েন্দা, ভৌতিক, মিথ, থ্রিলার, ও অ্যাড...",5,এই সংগ্রহটি তিনটি দুর্দান্ত ক্লাসিক উপন্যাস নি...,Rokomari,positive,2,"তিমিরপ্রেম,সপ্তআতঙ্ক,লর্নাডুন(পেপারব্যাক)তিনটি..."
6307,51194,Guinness World Records 2020 (Hardcover),Guinness World Records (Editor),"Foreign Language Books: Essay, Articles & Res...",5,ভাল বই,Rokomari,positive,2,GuinnessWorldRecords2020(Hardcover)
6308,35327,খুন (হার্ডকভার),আহমেদ ফারুক,সমকালীন উপন্যাস,5,সম্পূর্ণ প্রাপ্তবয়স্কদের বই। তবে দুর্দান্ত গল...,Rokomari,positive,2,খুন(হার্ডকভার)
6309,8262,একাত্তর এবং মুক্তিযোদ্ধা পরিবার (হার্ডকভার),আওলিয়া খানম,"মুক্তিযুদ্ধের ডায়েরি, চিঠি ও স্মৃতিচারণ",5,"এটি ""সাহিত্য কথা"" প্রকাশনী হতে প্রকাশিত একটি অ...",Rokomari,positive,2,একাত্তরএবংমুক্তিযোদ্ধাপরিবার(হার্ডকভার)


In [ ]:
book_count = df.groupby('Writer_Name')['Book_Name'].count().reset_index().sort_values('Book_Name', ascending=False).head(20).set_index('Writer_Name')
book_count

,Book_Name
Writer_Name,
হুমায়ূন আহমেদ,79
মুহম্মদ জাফর ইকবাল,69
রকিব হাসান,43
সুনীল গঙ্গোপাধ্যায় (নীললোহিত),36
কাজী আনোয়ার হোসেন,31
আনিসুল হক,23
সমরেশ মজুমদার,22
حكيم الامت مولانا اشرف علي تهانوي رح ( হাকীমুল উম্মত মাওলানা আশরাফ আলী থানভী রহ.),21
মোশতাক আহমেদ,21


In [ ]:
df1 = pd.read_csv('/content/drive/MyDrive/AI Project/zerin/user_info.csv', error_bad_lines = False)
df1 = df1.drop(columns=['Unnamed: 0'])
df1
calculateCount = df1[['book_id']].copy()
calculateCount

data = np.zeros((1, calculateCount.shape[0]))
x = 0;
for index, row in df1.iterrows():
  for x in row:
    if(x >=1 and x<= 5):
      data[0][index]+=1
df['ratings_count'] = data[0]

df1

,book_id,user1,user2,user3,user4,user5,user6,user7,user8,user9,...,user91,user92,user93,user94,user95,user96,user97,user98,user99,user100
0,96523,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,39118,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2,141682,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,...,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0
3,136770,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
4,82740,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,...,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6306,21943,0.0,5.0,5.0,0.0,0.0,5.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,5.0,5.0,0.0,0.0
6307,51194,0.0,0.0,0.0,5.0,0.0,5.0,5.0,0.0,0.0,...,0.0,5.0,0.0,5.0,0.0,5.0,0.0,0.0,0.0,5.0
6308,35327,5.0,5.0,5.0,5.0,0.0,5.0,5.0,5.0,5.0,...,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0
6309,8262,5.0,5.0,5.0,5.0,5.0,5.0,5.0,0.0,5.0,...,5.0,5.0,0.0,0.0,5.0,5.0,5.0,5.0,0.0,0.0


In [ ]:
print("# of NaN in each columns:", df.isnull().sum(), sep='\n')

# of NaN in each columns:
id                           0
Book_Name                    0
Writer_Name                 17
Category                     3
Rating                       0
Review                       0
Site                         0
sentiment                    0
label                        0
Book_name_without_spaces     0
ratings_count                0
dtype: int64


In [ ]:
df = df.dropna()
df = df.reset_index()

df

,index,id,Book_Name,Writer_Name,Category,Rating,Review,Site,sentiment,label,Book_name_without_spaces,ratings_count
0,0,96523,মুনতাখাব হাদীস,মাওলানা মোঃ সাখাওয়াত উল্লাহ,হাদিস ও সুন্নাত,1,(১) বই টি শত শত জাল হাদিসে ভরপুর । (২) দ্বীন ...,Rokomari,negative,0,মুনতাখাবহাদীস,98.0
1,1,39118,মেঘাচ্ছন্ন ঈমান (পেপারব্যাক),শাইখ আব্দুল মালিক আল কাসিম,"ঈমান, আক্বিদা ও তাওবাহ",1,"বই রিভিউ : মেঘাচ্ছন্ন ঈমান পাঠ অনুভূতি: ""একটি...",Rokomari,negative,0,মেঘাচ্ছন্নঈমান(পেপারব্যাক),27.0
2,2,141682,সামষ্টিক অর্থনীতি (পার্স কোর্স দ্বিতীয় বর্ষের...,সুকেশ চন্দ্র জোয়ারদার,ডিগ্রি,1,পড়তে চাই,Rokomari,negative,0,সামষ্টিকঅর্থনীতি(পার্সকোর্সদ্বিতীয়বর্ষের-৩য়পত্...,75.0
3,3,136770,ছাত্রলীগের ইতিহাস বাংলাদেশের ইতিহাস (১৯৪৮-১৯৭...,সাইফুর রহমান সোহাগ,প্রসঙ্গ: বাংলাদেশের রাজনীতি,1,ঠিক আছে,Rokomari,negative,0,ছাত্রলীগেরইতিহাসবাংলাদেশেরইতিহাস(১৯৪৮-১৯৭০প্রথ...,51.0
4,4,82740,"এসএসসি বিজ্ঞান মেইড ইজি প্রশ্নপত্র, সকল শিক্ষ...",পাঞ্জেরী সম্পাদনা পর্ষদ (সম্পাদক),নবম ও দশম (এসএসসি): বিজ্ঞান বিভাগ পাঠ্য সহায়...,1,এটি এসএসসি শিক্ষার্থীর 2020 এর জন্য একটি পরীক্...,Rokomari,negative,0,"এসএসসিবিজ্ঞানমেইডইজিপ্রশ্নপত্র,সকলশিক্ষাবোর্ড,...",77.0
...,...,...,...,...,...,...,...,...,...,...,...,...
6287,6306,21943,"তিমির প্রেম, সপ্ত আতঙ্ক, লর্না ডুন (পেপারব্যা...",আর. ডি. ব্ল্যাকমোর,"রহস্য, গোয়েন্দা, ভৌতিক, মিথ, থ্রিলার, ও অ্যাড...",5,এই সংগ্রহটি তিনটি দুর্দান্ত ক্লাসিক উপন্যাস নি...,Rokomari,positive,2,"তিমিরপ্রেম,সপ্তআতঙ্ক,লর্নাডুন(পেপারব্যাক)তিনটি...",33.0
6288,6307,51194,Guinness World Records 2020 (Hardcover),Guinness World Records (Editor),"Foreign Language Books: Essay, Articles & Res...",5,ভাল বই,Rokomari,positive,2,GuinnessWorldRecords2020(Hardcover),39.0
6289,6308,35327,খুন (হার্ডকভার),আহমেদ ফারুক,সমকালীন উপন্যাস,5,সম্পূর্ণ প্রাপ্তবয়স্কদের বই। তবে দুর্দান্ত গল...,Rokomari,positive,2,খুন(হার্ডকভার),93.0
6290,6309,8262,একাত্তর এবং মুক্তিযোদ্ধা পরিবার (হার্ডকভার),আওলিয়া খানম,"মুক্তিযুদ্ধের ডায়েরি, চিঠি ও স্মৃতিচারণ",5,"এটি ""সাহিত্য কথা"" প্রকাশনী হতে প্রকাশিত একটি অ...",Rokomari,positive,2,একাত্তরএবংমুক্তিযোদ্ধাপরিবার(হার্ডকভার),61.0


In [ ]:
df = df.drop(columns=['index'])

df

,id,Book_Name,Writer_Name,Category,Rating,Review,Site,sentiment,label,Book_name_without_spaces,ratings_count
0,96523,মুনতাখাব হাদীস,মাওলানা মোঃ সাখাওয়াত উল্লাহ,হাদিস ও সুন্নাত,1,(১) বই টি শত শত জাল হাদিসে ভরপুর । (২) দ্বীন ...,Rokomari,negative,0,মুনতাখাবহাদীস,98.0
1,39118,মেঘাচ্ছন্ন ঈমান (পেপারব্যাক),শাইখ আব্দুল মালিক আল কাসিম,"ঈমান, আক্বিদা ও তাওবাহ",1,"বই রিভিউ : মেঘাচ্ছন্ন ঈমান পাঠ অনুভূতি: ""একটি...",Rokomari,negative,0,মেঘাচ্ছন্নঈমান(পেপারব্যাক),27.0
2,141682,সামষ্টিক অর্থনীতি (পার্স কোর্স দ্বিতীয় বর্ষের...,সুকেশ চন্দ্র জোয়ারদার,ডিগ্রি,1,পড়তে চাই,Rokomari,negative,0,সামষ্টিকঅর্থনীতি(পার্সকোর্সদ্বিতীয়বর্ষের-৩য়পত্...,75.0
3,136770,ছাত্রলীগের ইতিহাস বাংলাদেশের ইতিহাস (১৯৪৮-১৯৭...,সাইফুর রহমান সোহাগ,প্রসঙ্গ: বাংলাদেশের রাজনীতি,1,ঠিক আছে,Rokomari,negative,0,ছাত্রলীগেরইতিহাসবাংলাদেশেরইতিহাস(১৯৪৮-১৯৭০প্রথ...,51.0
4,82740,"এসএসসি বিজ্ঞান মেইড ইজি প্রশ্নপত্র, সকল শিক্ষ...",পাঞ্জেরী সম্পাদনা পর্ষদ (সম্পাদক),নবম ও দশম (এসএসসি): বিজ্ঞান বিভাগ পাঠ্য সহায়...,1,এটি এসএসসি শিক্ষার্থীর 2020 এর জন্য একটি পরীক্...,Rokomari,negative,0,"এসএসসিবিজ্ঞানমেইডইজিপ্রশ্নপত্র,সকলশিক্ষাবোর্ড,...",77.0
...,...,...,...,...,...,...,...,...,...,...,...
6287,21943,"তিমির প্রেম, সপ্ত আতঙ্ক, লর্না ডুন (পেপারব্যা...",আর. ডি. ব্ল্যাকমোর,"রহস্য, গোয়েন্দা, ভৌতিক, মিথ, থ্রিলার, ও অ্যাড...",5,এই সংগ্রহটি তিনটি দুর্দান্ত ক্লাসিক উপন্যাস নি...,Rokomari,positive,2,"তিমিরপ্রেম,সপ্তআতঙ্ক,লর্নাডুন(পেপারব্যাক)তিনটি...",33.0
6288,51194,Guinness World Records 2020 (Hardcover),Guinness World Records (Editor),"Foreign Language Books: Essay, Articles & Res...",5,ভাল বই,Rokomari,positive,2,GuinnessWorldRecords2020(Hardcover),39.0
6289,35327,খুন (হার্ডকভার),আহমেদ ফারুক,সমকালীন উপন্যাস,5,সম্পূর্ণ প্রাপ্তবয়স্কদের বই। তবে দুর্দান্ত গল...,Rokomari,positive,2,খুন(হার্ডকভার),93.0
6290,8262,একাত্তর এবং মুক্তিযোদ্ধা পরিবার (হার্ডকভার),আওলিয়া খানম,"মুক্তিযুদ্ধের ডায়েরি, চিঠি ও স্মৃতিচারণ",5,"এটি ""সাহিত্য কথা"" প্রকাশনী হতে প্রকাশিত একটি অ...",Rokomari,positive,2,একাত্তরএবংমুক্তিযোদ্ধাপরিবার(হার্ডকভার),61.0


In [ ]:
print("# of NaN in each columns:", df.isnull().sum(), sep='\n')

# of NaN in each columns:
id                          0
Book_Name                   0
Writer_Name                 0
Category                    0
Rating                      0
Review                      0
Site                        0
sentiment                   0
label                       0
Book_name_without_spaces    0
ratings_count               0
dtype: int64


In [ ]:
# Melt the DataFrame to transform it to the desired for
ratings = pd.melt(df1, id_vars=['book_id'], var_name='User', value_name='Rating')
ratings.shape


(631100, 3)

In [ ]:

df.rename(columns={'id':'book_id'}, inplace = True)
df

,book_id,Book_Name,Writer_Name,Category,Rating,Review,Site,sentiment,label,Book_name_without_spaces,ratings_count
0,96523,মুনতাখাব হাদীস,মাওলানা মোঃ সাখাওয়াত উল্লাহ,হাদিস ও সুন্নাত,1,(১) বই টি শত শত জাল হাদিসে ভরপুর । (২) দ্বীন ...,Rokomari,negative,0,মুনতাখাবহাদীস,98.0
1,39118,মেঘাচ্ছন্ন ঈমান (পেপারব্যাক),শাইখ আব্দুল মালিক আল কাসিম,"ঈমান, আক্বিদা ও তাওবাহ",1,"বই রিভিউ : মেঘাচ্ছন্ন ঈমান পাঠ অনুভূতি: ""একটি...",Rokomari,negative,0,মেঘাচ্ছন্নঈমান(পেপারব্যাক),27.0
2,141682,সামষ্টিক অর্থনীতি (পার্স কোর্স দ্বিতীয় বর্ষের...,সুকেশ চন্দ্র জোয়ারদার,ডিগ্রি,1,পড়তে চাই,Rokomari,negative,0,সামষ্টিকঅর্থনীতি(পার্সকোর্সদ্বিতীয়বর্ষের-৩য়পত্...,75.0
3,136770,ছাত্রলীগের ইতিহাস বাংলাদেশের ইতিহাস (১৯৪৮-১৯৭...,সাইফুর রহমান সোহাগ,প্রসঙ্গ: বাংলাদেশের রাজনীতি,1,ঠিক আছে,Rokomari,negative,0,ছাত্রলীগেরইতিহাসবাংলাদেশেরইতিহাস(১৯৪৮-১৯৭০প্রথ...,51.0
4,82740,"এসএসসি বিজ্ঞান মেইড ইজি প্রশ্নপত্র, সকল শিক্ষ...",পাঞ্জেরী সম্পাদনা পর্ষদ (সম্পাদক),নবম ও দশম (এসএসসি): বিজ্ঞান বিভাগ পাঠ্য সহায়...,1,এটি এসএসসি শিক্ষার্থীর 2020 এর জন্য একটি পরীক্...,Rokomari,negative,0,"এসএসসিবিজ্ঞানমেইডইজিপ্রশ্নপত্র,সকলশিক্ষাবোর্ড,...",77.0
...,...,...,...,...,...,...,...,...,...,...,...
6287,21943,"তিমির প্রেম, সপ্ত আতঙ্ক, লর্না ডুন (পেপারব্যা...",আর. ডি. ব্ল্যাকমোর,"রহস্য, গোয়েন্দা, ভৌতিক, মিথ, থ্রিলার, ও অ্যাড...",5,এই সংগ্রহটি তিনটি দুর্দান্ত ক্লাসিক উপন্যাস নি...,Rokomari,positive,2,"তিমিরপ্রেম,সপ্তআতঙ্ক,লর্নাডুন(পেপারব্যাক)তিনটি...",33.0
6288,51194,Guinness World Records 2020 (Hardcover),Guinness World Records (Editor),"Foreign Language Books: Essay, Articles & Res...",5,ভাল বই,Rokomari,positive,2,GuinnessWorldRecords2020(Hardcover),39.0
6289,35327,খুন (হার্ডকভার),আহমেদ ফারুক,সমকালীন উপন্যাস,5,সম্পূর্ণ প্রাপ্তবয়স্কদের বই। তবে দুর্দান্ত গল...,Rokomari,positive,2,খুন(হার্ডকভার),93.0
6290,8262,একাত্তর এবং মুক্তিযোদ্ধা পরিবার (হার্ডকভার),আওলিয়া খানম,"মুক্তিযুদ্ধের ডায়েরি, চিঠি ও স্মৃতিচারণ",5,"এটি ""সাহিত্য কথা"" প্রকাশনী হতে প্রকাশিত একটি অ...",Rokomari,positive,2,একাত্তরএবংমুক্তিযোদ্ধাপরিবার(হার্ডকভার),61.0


In [ ]:
books=df
ratings=ratings
ratings

,book_id,User,Rating
0,96523,user1,1.0
1,39118,user1,0.0
2,141682,user1,0.0
3,136770,user1,1.0
4,82740,user1,1.0
...,...,...,...
631095,21943,user100,0.0
631096,51194,user100,5.0
631097,35327,user100,5.0
631098,8262,user100,0.0


In [ ]:
books.head()

,book_id,Book_Name,Writer_Name,Category,Rating,Review,Site,sentiment,label,Book_name_without_spaces,ratings_count
0,96523,মুনতাখাব হাদীস,মাওলানা মোঃ সাখাওয়াত উল্লাহ,হাদিস ও সুন্নাত,1,(১) বই টি শত শত জাল হাদিসে ভরপুর । (২) দ্বীন ...,Rokomari,negative,0,মুনতাখাবহাদীস,98.0
1,39118,মেঘাচ্ছন্ন ঈমান (পেপারব্যাক),শাইখ আব্দুল মালিক আল কাসিম,"ঈমান, আক্বিদা ও তাওবাহ",1,"বই রিভিউ : মেঘাচ্ছন্ন ঈমান পাঠ অনুভূতি: ""একটি...",Rokomari,negative,0,মেঘাচ্ছন্নঈমান(পেপারব্যাক),27.0
2,141682,সামষ্টিক অর্থনীতি (পার্স কোর্স দ্বিতীয় বর্ষের...,সুকেশ চন্দ্র জোয়ারদার,ডিগ্রি,1,পড়তে চাই,Rokomari,negative,0,সামষ্টিকঅর্থনীতি(পার্সকোর্সদ্বিতীয়বর্ষের-৩য়পত্...,75.0
3,136770,ছাত্রলীগের ইতিহাস বাংলাদেশের ইতিহাস (১৯৪৮-১৯৭...,সাইফুর রহমান সোহাগ,প্রসঙ্গ: বাংলাদেশের রাজনীতি,1,ঠিক আছে,Rokomari,negative,0,ছাত্রলীগেরইতিহাসবাংলাদেশেরইতিহাস(১৯৪৮-১৯৭০প্রথ...,51.0
4,82740,"এসএসসি বিজ্ঞান মেইড ইজি প্রশ্নপত্র, সকল শিক্ষ...",পাঞ্জেরী সম্পাদনা পর্ষদ (সম্পাদক),নবম ও দশম (এসএসসি): বিজ্ঞান বিভাগ পাঠ্য সহায়...,1,এটি এসএসসি শিক্ষার্থীর 2020 এর জন্য একটি পরীক্...,Rokomari,negative,0,"এসএসসিবিজ্ঞানমেইডইজিপ্রশ্নপত্র,সকলশিক্ষাবোর্ড,...",77.0


In [ ]:

rating_books = pd.merge(ratings, books, on='book_id')
rating_books.head()
rating_books = rating_books.drop(columns=['Rating_y'])
rating_books.rename(columns={'Rating_x':'Rating'}, inplace = True)
######rating_books_sample = rating_books.sample(frac=.01, random_state=1)

# Shape of the sample data
#rating_books_sample
rating_books
#rating_books.shape

,book_id,User,Rating,Book_Name,Writer_Name,Category,Review,Site,sentiment,label,Book_name_without_spaces,ratings_count
0,96523,user1,1.0,মুনতাখাব হাদীস,মাওলানা মোঃ সাখাওয়াত উল্লাহ,হাদিস ও সুন্নাত,(১) বই টি শত শত জাল হাদিসে ভরপুর । (২) দ্বীন ...,Rokomari,negative,0,মুনতাখাবহাদীস,98.0
1,96523,user2,1.0,মুনতাখাব হাদীস,মাওলানা মোঃ সাখাওয়াত উল্লাহ,হাদিস ও সুন্নাত,(১) বই টি শত শত জাল হাদিসে ভরপুর । (২) দ্বীন ...,Rokomari,negative,0,মুনতাখাবহাদীস,98.0
2,96523,user3,1.0,মুনতাখাব হাদীস,মাওলানা মোঃ সাখাওয়াত উল্লাহ,হাদিস ও সুন্নাত,(১) বই টি শত শত জাল হাদিসে ভরপুর । (২) দ্বীন ...,Rokomari,negative,0,মুনতাখাবহাদীস,98.0
3,96523,user4,1.0,মুনতাখাব হাদীস,মাওলানা মোঃ সাখাওয়াত উল্লাহ,হাদিস ও সুন্নাত,(১) বই টি শত শত জাল হাদিসে ভরপুর । (২) দ্বীন ...,Rokomari,negative,0,মুনতাখাবহাদীস,98.0
4,96523,user5,1.0,মুনতাখাব হাদীস,মাওলানা মোঃ সাখাওয়াত উল্লাহ,হাদিস ও সুন্নাত,(১) বই টি শত শত জাল হাদিসে ভরপুর । (২) দ্বীন ...,Rokomari,negative,0,মুনতাখাবহাদীস,98.0
...,...,...,...,...,...,...,...,...,...,...,...,...
629195,58654,user96,0.0,রহস্যের দ্বীপ (হার্ডকভার),জুল ভার্ন,শিশু-কিশোর উপন্যাস,অসাধারন😍😍,Rokomari,positive,2,রহস্যেরদ্বীপ(হার্ডকভার),51.0
629196,58654,user97,5.0,রহস্যের দ্বীপ (হার্ডকভার),জুল ভার্ন,শিশু-কিশোর উপন্যাস,অসাধারন😍😍,Rokomari,positive,2,রহস্যেরদ্বীপ(হার্ডকভার),51.0
629197,58654,user98,5.0,রহস্যের দ্বীপ (হার্ডকভার),জুল ভার্ন,শিশু-কিশোর উপন্যাস,অসাধারন😍😍,Rokomari,positive,2,রহস্যেরদ্বীপ(হার্ডকভার),51.0
629198,58654,user99,0.0,রহস্যের দ্বীপ (হার্ডকভার),জুল ভার্ন,শিশু-কিশোর উপন্যাস,অসাধারন😍😍,Rokomari,positive,2,রহস্যেরদ্বীপ(হার্ডকভার),51.0


In [ ]:
# Create Item-user matrix using pivot_table()
rating_books_pivot = rating_books.pivot_table(index='Book_Name', columns='User', values='Rating').fillna(0)

# Show top-5 records
rating_books_pivot
rating_books_pivot

User,user1,user10,user100,user11,user12,user13,user14,user15,user16,user17,...,user90,user91,user92,user93,user94,user95,user96,user97,user98,user99
Book_Name,,,,,,,,,,,,,,,,,,,,,
Florence Nursing Book Series for 1st Year B.Sc in Nursing (Post Basic/Post Basic Public Health Nursing) Students,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0
আরবি রস (পেপারব্যাক) আরবি ভাষা ও সাহিত্যের মজা।,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
উত্তরণ প্রকাশনীর ৫ টি বইয়ের প্যাকেজ (হার্ডকভার),0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
এরাবিয়ান নাইটস এন্ড ডেইজ (পুরস্কারপ্রাপ্ত লেখকদের বই),5.0,5.0,5.0,0.0,5.0,5.0,5.0,5.0,5.0,0.0,...,5.0,0.0,5.0,5.0,5.0,5.0,0.0,0.0,5.0,5.0
কফিমেকার (গোয়েন্দা কাহিনি) (হার্ডকভার),0.0,5.0,5.0,0.0,5.0,0.0,0.0,0.0,5.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
হে বোন কুরআন তোমাকে যা বলেছে (নীল কভার),4.0,0.0,2.0,0.0,1.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
হোয়াই কাস্টমার’স ডোন্ট ডু হোয়াট ইউ ওয়ান্ট দেম টু ডু,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
হ্যাপি ম্যারিড লাইফ,5.0,5.0,5.0,5.0,5.0,0.0,5.0,5.0,5.0,5.0,...,5.0,0.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0


In [ ]:
rating_books.shape
rating_books_pivot.shape

(6292, 100)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import mean_absolute_error, mean_squared_error
from scipy.sparse import csr_matrix

# ... (your existing code)

def get_user_index(user_id):
    return rating_books[rating_books["User"].astype(str) == user_id].index.tolist()[0]

def get_recommendations_and_metrics(user_id_input, n=5):
    # Convert pivot table to a sparse matrix
    rating_matrix = csr_matrix(rating_books_pivot.values)

    # Initialize the Nearest Neighbors model
    knn_model = NearestNeighbors(metric='cosine', algorithm='brute')
    knn_model.fit(rating_matrix)

    try:
        user_index = get_user_index(user_id_input)
        distances, indices = knn_model.kneighbors(rating_matrix[user_index], n_neighbors=n+1)
        # Exclude the user's own index (top result is the user itself)
        indices = indices.squeeze()[1:]
        recommendations = rating_books.iloc[indices][["Book_Name", "book_id"]]

        # Drop duplicate book_ids and keep at least 5 unique recommendations
        unique_recommendations = recommendations.drop_duplicates(subset='book_id')

        # Keep adding unique books until you have at least 5 recommendations
        while len(unique_recommendations) < n and len(recommendations) > 0:
            recommendations = recommendations[~recommendations['book_id'].isin(unique_recommendations['book_id'])]
            unique_recommendations = pd.concat([unique_recommendations, recommendations.head(1)])

        unique_recommendations = unique_recommendations.head(n)

        print(unique_recommendations)

    except ValueError:
        print("Please enter a valid user ID.")



# Example: Get recommendations for a user with user_id "user28"
user_id_input = input("Enter user ID (e.g., user28): ")
get_recommendations_and_metrics(user_id_input)


Enter user ID (e.g., user28): user34
                                              Book_Name  book_id
5588   How to Teach English (Paperback)  An Elite bo...    43614
3852   সোফির জগৎ (হার্ডকভার)  আন্তর্জাতিক পুরস্কার প...    38660
686             দিওয়ান-ই-শামস্‌ তাবরিজ (হার্ডকভার)         56632
2685                         মুসলিম শরীফ (১ম-৬ষ্ঠ খণ্ড)   157930
958    গুচ্ছ ফাইনাল মডেল টেস্ট (পেপারব্যাক)  গুচ্ছ ব...   130633


In [ ]:
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import mean_absolute_error, mean_squared_error
from scipy.sparse import csr_matrix
from surprise import Dataset, Reader, KNNWithZScore
from surprise.model_selection import train_test_split
from surprise import accuracy

def evaluate_knn_model():
    try:
        # Assuming 'User', 'book_id', 'Rating' are the columns in your DataFrame
        reader = Reader(rating_scale=(1, 5))
        data = Dataset.load_from_df(rating_books[['User', 'book_id', 'Rating']], reader)

        # Split the data into train and test sets
        trainset, testset = train_test_split(data, test_size=0.2)

        # Use the k-NN with Z-score model
        model = KNNWithZScore(k=5, sim_options={'user_based': True})
        model.fit(trainset)

        # Make predictions on the test set
        predictions = model.test(testset)

        # Evaluate the model using Surprise's accuracy functions
        accuracy.mae(predictions, verbose=True)
        accuracy.rmse(predictions, verbose=True)

    except ValueError:
        print("Error evaluating the k-NN model.")

# Example: Evaluate the k-NN model
evaluate_knn_model()

Computing the msd similarity matrix...
Done computing similarity matrix.
MAE:  1.4325
RMSE: 1.8295
